In [ ]:
spark.stop()

## DLRM Categorify

In [1]:
###### Start spark job ######
import init

from pyrecdp.data_processor import *
from pyrecdp.utils import *

import logging
from timeit import default_timer as timer
import os
from pyspark import *
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np
import pandas as pd

# Define Schema
LABEL_COL = 0
INT_COLS = list(range(1, 14))
CAT_COLS = list(range(14, 40))
label_fields = [StructField('_c%d' % LABEL_COL, IntegerType())]
int_fields = [StructField('_c%d' % i, IntegerType()) for i in INT_COLS]
str_fields = [StructField('_c%d' % i, StringType()) for i in CAT_COLS]
schema = StructType(label_fields + int_fields + str_fields)

to_be_categorified = [23, 35, 14, 33]

def categorifyAllFeatures(df, proc, output_name="categorified", gen_dict=False, enable_freqlimit=False):
    dict_dfs = []
    to_categorify_cols = ['_c%d' % i for i in CAT_COLS]
    #to_categorify_cols = ['_c%d' % i for i in to_be_categorified]
    if gen_dict:
        # only call below function when target dicts were not pre-prepared        
        op_gen_dict = GenerateDictionary(to_categorify_cols, isParquet=False)
        proc.reset_ops([op_gen_dict])
        t1 = timer()
        dict_dfs = proc.generate_dicts(df)
        t2 = timer()
        print("Generate Dictionary took %.3f" % (t2 - t1))
    else:
        # or we can simply load from pre-gened
        dict_dfs = [{'col_name': name, 'dict': proc.spark.read.parquet(
            "%s/%s/%s/%s" % (proc.path_prefix, proc.current_path, proc.dicts_path, name))} for name in to_categorify_cols]    

    if enable_freqlimit:
        dict_dfs = [{'col_name': dict_df['col_name'], 'dict': dict_df['dict'].filter('count >= 15')} for dict_df in dict_dfs]

    # start to do categorify
    op_categorify = Categorify(to_categorify_cols, dict_dfs=dict_dfs)
    op_fillna_for_categorified = FillNA(to_categorify_cols, 0)
    proc.append_ops([op_categorify, op_fillna_for_categorified])
    t1 = timer()
    df = proc.transform(df, name=output_name)
    t2 = timer()
    print("Categorify took %.3f" % (t2 - t1))
    
    return df

/mnt/nvme2/chendi/BlueWhale/recdp


In [2]:
path_prefix = "hdfs://"
current_path = "/dlrm/"
csv_folder = "/dlrm/csv_raw_data"
#path = os.path.join(path_prefix, file)

scala_udf_jars = "/mnt/nvme2/chendi/BlueWhale/recdp/ScalaProcessUtils/target/core-0.1.0-jar-with-dependencies.jar"

##### 1. Start spark and initialize data processor #####
t0 = timer()
spark = SparkSession\
    .builder\
    .master('yarn')\
    .appName("DLRM")\
    .config("spark.executor.memory", "20g")\
    .config("spark.executor.memoryOverhead", "3g")\
    .config("spark.cleaner.periodicGC.interval", "10min")\
    .config("spark.driver.extraClassPath", f"{scala_udf_jars}")\
    .config("spark.executor.extraClassPath", f"{scala_udf_jars}")\
    .getOrCreate()

#files = ["day_%d" % i for i in range(0, 24)]
files = ["day_0"]
file_names = [os.path.join(path_prefix, csv_folder, filename) for filename in files]

proc = DataProcessor(spark, path_prefix, current_path=current_path, shuffle_disk_capacity="1200GB")
df = spark.read.schema(schema).option('sep', '\t').csv(file_names)
#df = spark.read.parquet("/dlrm/categorified_stage1")
df = categorifyAllFeatures(df, proc, output_name="dlrm_categorified", gen_dict=False, enable_freqlimit=True)

per core memory size is 5.000 GB and shuffle_disk maximum capacity is 1200.000 GB
bhj total threshold is 67.109 M rows, one bhj threshold is 30.000 M rows, flush_threshold is 960.000 GB
('_c19', DataFrame[dict_col: string, count: bigint, dict_col_id: int], 3)
('_c30', DataFrame[dict_col: string, count: bigint, dict_col_id: int], 3)
('_c26', DataFrame[dict_col: string, count: bigint, dict_col_id: int], 10)
('_c32', DataFrame[dict_col: string, count: bigint, dict_col_id: int], 14)
('_c39', DataFrame[dict_col: string, count: bigint, dict_col_id: int], 34)
('_c22', DataFrame[dict_col: string, count: bigint, dict_col_id: int], 62)
('_c38', DataFrame[dict_col: string, count: bigint, dict_col_id: int], 96)
('_c29', DataFrame[dict_col: string, count: bigint, dict_col_id: int], 103)
('_c31', DataFrame[dict_col: string, count: bigint, dict_col_id: int], 967)
('_c21', DataFrame[dict_col: string, count: bigint, dict_col_id: int], 1381)
('_c27', DataFrame[dict_col: string, count: bigint, dict_col_i

History Log

* per core memory size is 5.000 GB and shuffle_disk maximum capacity is 1200.000 GB
* Generate Dictionary took 2373.706
* categorify threshold is 50.000 M rows, flush_threshold is 960.000 GB
* ('_c19', DataFrame[dict_col: string, dict_col_id: int], 3)
* ('_c30', DataFrame[dict_col: string, dict_col_id: int], 3)
* ('_c26', DataFrame[dict_col: string, dict_col_id: int], 10)
* ('_c32', DataFrame[dict_col: string, dict_col_id: int], 14)
* ('_c39', DataFrame[dict_col: string, dict_col_id: int], 36)
* ('_c22', DataFrame[dict_col: string, dict_col_id: int], 63)
* ('_c38', DataFrame[dict_col: string, dict_col_id: int], 108)
* ('_c29', DataFrame[dict_col: string, dict_col_id: int], 154)
* ('_c31', DataFrame[dict_col: string, dict_col_id: int], 976)
* ('_c21', DataFrame[dict_col: string, dict_col_id: int], 1543)
* ('_c27', DataFrame[dict_col: string, dict_col_id: int], 2208)
* ('_c20', DataFrame[dict_col: string, dict_col_id: int], 7122)
* ('_c17', DataFrame[dict_col: string, dict_col_id: int], 7424)
* ('_c28', DataFrame[dict_col: string, dict_col_id: int], 11938)
* ('_c37', DataFrame[dict_col: string, dict_col_id: int], 12973)
* ('_c16', DataFrame[dict_col: string, dict_col_id: int], 17295)
* ('_c18', DataFrame[dict_col: string, dict_col_id: int], 20265)
* ('_c15', DataFrame[dict_col: string, dict_col_id: int], 39060)
* ('_c25', DataFrame[dict_col: string, dict_col_id: int], 405282)
* ('_c36', DataFrame[dict_col: string, dict_col_id: int], 590151)
* ('_c24', DataFrame[dict_col: string, dict_col_id: int], 3067955)
* ('_c34', DataFrame[dict_col: string, dict_col_id: int], 40790947)
* ('_c23', DataFrame[dict_col: string, dict_col_id: int], 130229466)
* ('_c35', DataFrame[dict_col: string, dict_col_id: int], 187188509)
* ('_c14', DataFrame[dict_col: string, dict_col_id: int], 227605431)
* ('_c33', DataFrame[dict_col: string, dict_col_id: int], 292775613)
* _c19 will do bhj
* _c30 will do bhj
* _c26 will do bhj
* _c32 will do bhj
* _c39 will do bhj
* _c22 will do bhj
* _c38 will do bhj
* _c29 will do bhj
* _c31 will do bhj
* _c21 will do bhj
* _c27 will do bhj
* _c20 will do bhj
* _c17 will do bhj
* _c28 will do bhj
* _c37 will do bhj
* _c16 will do bhj
* _c18 will do bhj
* _c15 will do bhj
* _c25 will do bhj
* _c36 will do bhj
* _c24 will do bhj
* _c34 will do bhj
* etstimated_to_shuffle_size for _c23 is 749.453 GB, will do smj
* etstimated_to_shuffle_size for _c35 is 725.014 GB, will do smj and spill to disk
* etstimated_to_shuffle_size for _c14 is 700.576 GB, will do smj
* etstimated_to_shuffle_size for _c33 is 676.137 GB, will do smj and spill to disk
* do bhj to _c19
* do bhj to _c30
* do bhj to _c26
* do bhj to _c32
* do bhj to _c39
* do bhj to _c22
* do bhj to _c38
* do bhj to _c29
* do bhj to _c31
* do bhj to _c21
* do bhj to _c27
* do bhj to _c20
* do bhj to _c17
* do bhj to _c28
* do bhj to _c37
* do bhj to _c16
* do bhj to _c18
* do bhj to _c15
* do bhj to _c25
* do bhj to _c36
* do bhj to _c24
* do bhj to _c34
* do smj to _c23
* do smj to _c35
* do smj to _c14
* do smj to _c33
* Categorify took 12443.532